# 案例说明：分支渠网闸门控制

本案例模拟了一个基础的分支渠网系统。该系统的核心是一个由单座闸门（Gate）控制、连接上下游两条渠道（Reach）的水力学模型。

这个案例的主要目的是：
- 演示如何通过JSON配置文件来定义一个水网的拓扑结构、物理参数和边界条件。
- 演示如何通过配置`disturbances`（扰动）来模拟仿真过程中的人工干预或突发事件。
- 作为后续更复杂案例重构的基础模板。

### 1. 导入库并加载配置

In [ ]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from swp.simulation_identification.physical_objects.st_venant_reach import StVenantReach
from swp.simulation_identification.hydro_nodes.gate_node import GateNode
from swp.core_engine.solver.network_solver import NetworkSolver

CONFIG_PATH = 'examples/branched_network.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 定义仿真运行函数

In [ ]:
def run_hydrodynamic_network_from_config(cfg):
    """根据JSON配置文件搭建并运行水力网络仿真。"""
    print(f"--- 开始从配置文件加载仿真: {CONFIG_PATH} ---")
    
    COMPONENT_CLASS_MAP = {
        "StVenantReach": StVenantReach,
        "GateNode": GateNode,
    }

    # 1. 定义仿真参数
    settings = cfg['simulation_settings']
    sim_dt = settings['dt']
    sim_theta = settings['theta']
    num_steps = settings['num_steps']
    disturbances = {d['time_step']: d for d in cfg.get('disturbances', [])}

    # 2. 动态实例化组件
    print("\n--- 正在实例化组件 ---")
    components = {}
    for comp_conf in cfg['components']:
        comp_id, comp_type, params = comp_conf['id'], comp_conf['type'], comp_conf['params']
        CompClass = COMPONENT_CLASS_MAP[comp_type]
        if comp_type == "StVenantReach":
            params['initial_H'] = np.full(params['num_points'], params.pop('initial_H'))
            params['initial_Q'] = np.full(params['num_points'], params.pop('initial_Q'))
        instance = CompClass(name=comp_id, **params)
        if 'initial_opening' in comp_conf: instance.set_opening(comp_conf['initial_opening'])
        components[comp_id] = instance
        print(f"  - 已创建 {comp_type}: {comp_id}")

    # 3. 创建并配置求解器
    solver = NetworkSolver(dt=sim_dt, theta=sim_theta)
    for comp in components.values(): solver.add_component(comp)

    # 4. 动态连接组件
    print("\n--- 正在连接组件 ---")
    for conn in cfg['connections']:
        from_obj, to_obj = components[conn['from']], components[conn['to']]
        if isinstance(from_obj, StVenantReach) and isinstance(to_obj, GateNode):
            to_obj.link_to_reaches(up_obj=from_obj, down_obj=None)
            to_obj.upstream_idx = -1
        elif isinstance(from_obj, GateNode) and isinstance(to_obj, StVenantReach):
            from_obj.link_to_reaches(up_obj=from_obj.upstream_obj, down_obj=to_obj)
            from_obj.downstream_idx = 0
    print("  - 连接已建立。")

    # 5. 设置边界条件
    print("\n--- 正在设置边界条件 ---")
    for bc in cfg.get('boundary_conditions', []):
        solver.add_boundary_condition(components[bc['component_id']], bc['variable'], bc['index'], lambda t, v=bc['value']: v)
        print(f"  - 已为 {bc['component_id']} 设置边界条件")

    # 6. 运行仿真
    print("\n--- 开始水力学仿真 ---")
    results = {"time": [], "components": {cid: {"H": [], "Q": []} for cid, c in components.items() if isinstance(c, StVenantReach)}}
    for i in range(num_steps):
        current_time = i * solver.dt
        if i in disturbances:
            event = disturbances[i]
            print(f"\n!!! 在 t={current_time:.1f}s 时应用扰动: {event['component_id']}.{event['action']}({event['value']}) !!!\n")
            getattr(components[event['component_id']], event['action'])(event['value'])
        solver.step(current_time)
        results["time"].append(current_time)
        for comp_id, comp_data in results["components"].items():
            reach = components[comp_id]
            num_pts, mid_pt = len(reach.H), len(reach.H) // 2
            comp_data["H"].append([float(reach.H[0]), float(reach.H[mid_pt]), float(reach.H[-1])])
            comp_data["Q"].append([float(reach.Q[0]), float(reach.Q[mid_pt]), float(reach.Q[-1])])
    print("\n--- 仿真结束 ---")
    
    # 7. 保存结果到JSON文件
    output_path = os.path.join(os.path.dirname(CONFIG_PATH), "branched_network_output.json")
    with open(output_path, 'w') as f: json.dump(results, f, indent=4)
    print(f"\n--- 结果已保存至 {output_path} ---")
    return results

### 3. 运行仿真并可视化结果

In [ ]:
results = run_hydrodynamic_network_from_config(config)

def plot_results(res_data):
    reach_id = list(res_data['components'].keys())[0] # 假设只有一个reach
    reach_results = res_data['components'][reach_id]
    df = pd.DataFrame(reach_results, index=res_data['time'])
    
    fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
    fig.suptitle(f'渠段 {reach_id} 仿真结果', fontsize=16)
    
    # 绘制水位 H
    h_df = pd.DataFrame(df['H'].tolist(), index=df.index, columns=['上游端', '中点', '下游端'])
    h_df.plot(ax=axes[0], style=['-', '--', '-.'])
    axes[0].set_title('水位 (H)')
    axes[0].set_ylabel('水位 (m)')
    axes[0].grid(True)
    axes[0].legend()
    
    # 绘制流量 Q
    q_df = pd.DataFrame(df['Q'].tolist(), index=df.index, columns=['上游端', '中点', '下游端'])
    q_df.plot(ax=axes[1], style=['-', '--', '-.'])
    axes[1].set_title('流量 (Q)')
    axes[1].set_ylabel('流量 (m³/s)')
    axes[1].set_xlabel('时间 (s)')
    axes[1].grid(True)
    axes[1].legend()
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

plot_results(results)